**Import các thư viện cần thiết**

In [1]:
import re
import math
import numpy as np
from scipy.spatial import distance

**Tập dữ liệu văn bản/tài liệu mẫu và truy vấn**

In [2]:
# Tập văn bản/tài liệu (D)
docs = [
    'data mining is awesome. data mining helps to find frequent itemsets in database.',
    'information retrieval is cool. information retrieval helps to search data quickly.',
    'natural language processing is interesting. it helps computer to better understand text.'
];

# Truy vấn (q)
query = 'information retrieval'

# Để cho tiện lợi trong việc xử lý chúng ta sẽ gán câu truy vấn là 1 tài liệu cuối cùng
docs.append(query)

# Làm sạch văn bản - xóa các ký tự đặc biệt, dấu câu, v.v.
docs = [re.sub('\W+',' ', doc) for doc in docs] 

doc_len = len(docs)

**Đọc từng tài liệu/văn bản và thực hiện tách từ - gán id cho các token và xây dựng tập từ vựng**

**Ngoài ra chúng ta cũng cần xác định thêm các thông tin khác, bao gồm:**

*   **Số lần xuất hiện của mỗi token/từ trong mỗi văn bản (term frequency)**
*   **Số lượng văn bản mà mỗi token/từ xuất hiện (xét trong toàn tập văn bản)**




In [3]:
# Cấu trúc dữ liệu dạng dictionary -> <key: id, value: token>
id_token_dict = {}

# Cấu trúc dữ liệu dạng dictionary -> <key: token, value: id>
token_id_dict = {}

# Cấu trúc dữ liệu dạng dictionary -> <key: doc_id, value: [id_token_xuất_hiện_1, id_token_xuất_hiện_2, v.v. ]>
doc_id_token_ids_dict = {}

# Cấu trúc dữ liệu dạng dictionary -> <key: token_id, value: <key: doc_id, value: token_freq>
token_id_doc_id_term_freq_dict = {}

# Cấu trúc dữ liệu dạng dictionary -> <key: doc_id, value: <key: token_id, value: token_freq>
doc_id_token_id_term_freq_dict = {}

# Cấu trúc dữ liệu dạng dictionary -> <key: token_id, value: [doc_id_token_xuất_hiện_1, doc_id_token_xuất_hiện_2, v.v.]>
token_id_doc_ids_dict = {}

# Gán id của token đầu tiên là [0]
token_id = 0

# Duyệt qua từng tài liệu/văn bản có trong mảng
for doc_idx, doc in enumerate(docs):
  # Tách các từ riêng biệt trong tài liệu/văn bản bằng khoảng trắng " "
  tokens = doc.split(' ')
  
  # Khởi tạo danh sách các token_ids xuất hiện trong tài liệu/văn bản
  doc_token_ids = []

  # Đọc qua từng token trong mỗi tài liệu/văn bản
  for token in tokens:
    # Kiểm tra chiều dài từ khóa > 0
    if len(token) > 0:
      if token not in token_id_dict.keys():
        token_id_dict[token] = token_id
        id_token_dict[token_id] = token
        # Tăng id của token tiếp theo lên 1
        token_id+=1

      target_token_id = token_id_dict[token]

      # Đếm số lượng mà token này xuất hiện trong doc_idx
      if target_token_id not in token_id_doc_id_term_freq_dict.keys():
        token_id_doc_id_term_freq_dict[target_token_id] = {}
        token_id_doc_id_term_freq_dict[target_token_id][doc_idx] = 1
      else:
        if doc_idx not in token_id_doc_id_term_freq_dict[target_token_id].keys():
          token_id_doc_id_term_freq_dict[target_token_id][doc_idx] = 1
        else:
          token_id_doc_id_term_freq_dict[target_token_id][doc_idx] += 1
      
      if doc_idx not in doc_id_token_id_term_freq_dict.keys():
        doc_id_token_id_term_freq_dict[doc_idx] = {}
        doc_id_token_id_term_freq_dict[doc_idx][target_token_id] = 1
      else:
        if token_id not in doc_id_token_id_term_freq_dict[doc_idx].keys():
          doc_id_token_id_term_freq_dict[doc_idx][target_token_id] = 1
        else:
          doc_id_token_id_term_freq_dict[doc_idx][target_token_id] += 1

      # Gán doc_idx vào danh sách mà token này xuất hiện
      if target_token_id not in token_id_doc_ids_dict.keys():
        token_id_doc_ids_dict[target_token_id] = [doc_idx]
      else:
        if doc_idx not in token_id_doc_ids_dict[target_token_id]:
          token_id_doc_ids_dict[target_token_id].append(doc_idx)

      doc_token_ids.append(token_id_dict[token])
  doc_id_token_ids_dict[doc_idx] = doc_token_ids

# Xuất ra màn hình kích thước tập từ vựng (vocabulary)
vocab_size = len(id_token_dict.keys())
print('Kích thước tập từ vựng: [{}]'.format(vocab_size))
print('Tập từ vựng (V):')
print(id_token_dict)
print('---')
print('Thông tin về tầng số xuất hiện của từ trong mỗi tài liệu/văn bản:')
print(token_id_doc_id_term_freq_dict)
print(doc_id_token_id_term_freq_dict)
print('---')
print('Thông tin về từ - danh sách các tài liệu/văn bản mà nó xuất hiện:')
print(token_id_doc_ids_dict)

Kích thước tập từ vựng: [25]
Tập từ vựng (V):
{0: 'data', 1: 'mining', 2: 'is', 3: 'awesome', 4: 'helps', 5: 'to', 6: 'find', 7: 'frequent', 8: 'itemsets', 9: 'in', 10: 'database', 11: 'information', 12: 'retrieval', 13: 'cool', 14: 'search', 15: 'quickly', 16: 'natural', 17: 'language', 18: 'processing', 19: 'interesting', 20: 'it', 21: 'computer', 22: 'better', 23: 'understand', 24: 'text'}
---
Thông tin về tầng số xuất hiện của từ trong mỗi tài liệu/văn bản:
{0: {0: 2, 1: 1}, 1: {0: 2}, 2: {0: 1, 1: 1, 2: 1}, 3: {0: 1}, 4: {0: 1, 1: 1, 2: 1}, 5: {0: 1, 1: 1, 2: 1}, 6: {0: 1}, 7: {0: 1}, 8: {0: 1}, 9: {0: 1}, 10: {0: 1}, 11: {1: 2, 3: 1}, 12: {1: 2, 3: 1}, 13: {1: 1}, 14: {1: 1}, 15: {1: 1}, 16: {2: 1}, 17: {2: 1}, 18: {2: 1}, 19: {2: 1}, 20: {2: 1}, 21: {2: 1}, 22: {2: 1}, 23: {2: 1}, 24: {2: 1}}
{0: {0: 1, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1}, 1: {11: 1, 12: 1, 2: 1, 13: 1, 4: 1, 5: 1, 14: 1, 0: 1, 15: 1}, 2: {16: 1, 17: 1, 18: 1, 2: 1, 19: 1, 20: 1, 4: 1, 2

**Thực hiện tính TF-IDF và chuyển đổi các tài liệu/văn bản về dạng TF-IDF vector**

**Để xác định được TF-IDF của một từ (i) cho một văn bản (j), chúng ta cần xác định ($tf_{ij}$) và ($idf_{i}$), với:**
$$tf_{ij}=\frac{f_{ij}}{max_i(f_{ij})}$$
$$idf_{i}=log_{2}(\frac{n}{df_{i}})$$
**Trong đó**,
*   ($max(f_{ij})$): là tầng số xuất hiện cao nhất của một từ trong văn bản (j)
*   (n): là tổng số lượng tài liệu/văn bản [doc_len]
*   ($df_{i}$): là số lượng tài liệu/văn bản có từ khóa/token (i) xuất hiện



In [4]:
# Viết hàm xác định từ/token xuất hiện nhiều nhất trong 1 văn bản (d) và số lần xuất hiện
def find_max_freq_token(doc_idx):
  max_token_id = -1
  max_freq = 0
  for token_id in doc_id_token_id_term_freq_dict[doc_idx].keys():
    if doc_id_token_id_term_freq_dict[doc_idx][token_id] > max_freq:
      max_token_id = token_id
      max_freq = doc_id_token_id_term_freq_dict[doc_idx][token_id]
  return (max_token_id, max_freq)

# Viết hàm tính trọng số TF-IDF của 1 từ/token (t) và 1 văn bản (d)
def calc_tfidf(token_id, doc_idx):
  (max_token_id, max_freq) = find_max_freq_token(doc_idx)
  tf = token_id_doc_id_term_freq_dict[token_id][doc_idx] / max_freq
  df = len(token_id_doc_ids_dict[token_id])
  idf = math.log((doc_len / df), 2)
  return tf * idf

# Cấu trúc dữ liệu dạng dictionary -> <key: doc_id, value: tfidf_encoded_vector>
doc_id_tfidf_encoded_vector_dict = {}

# Duyệt qua từng tài liệu/văn bản có trong mảng
for doc_idx in doc_id_token_ids_dict.keys():
  # Khởi tạo một vector có số chiều = vocab_size và các giá trị [0] -> [0, 0, 0, ...]
  tfidf_encoded_vector = np.zeros(vocab_size)

  # Danh sách các token_ids xuất hiện trong tài liệu/văn bản
  doc_token_ids = doc_id_token_ids_dict[doc_idx]
  
  # Duyệt qua từng token xuất hiện trong tài liệu/văn bản
  for token_id in doc_token_ids:
    # Gán vị trí - cột/chiều thứ (i) của vector tính giá trị TF-IDF - tương đương với vị trí xuất hiện của token
    tfidf_encoded_vector[token_id] = calc_tfidf(token_id, doc_idx)
  doc_id_tfidf_encoded_vector_dict[doc_idx] = tfidf_encoded_vector

# TFIDF encode của truy vấn (q) là tài liệu cuối cùng
query_tfidf_encoded_vector = doc_id_tfidf_encoded_vector_dict[doc_len-1]

# Xóa query đã được mã hóa thành dạng tfidf vector ra khỏi danh sách
del doc_id_tfidf_encoded_vector_dict[doc_len-1]

print('Các tài liệu được chuyển đổi về dạng tf-idf vectors:')
for doc_idx in doc_id_tfidf_encoded_vector_dict.keys():
  doc_tfidf_encoded_vector = doc_id_tfidf_encoded_vector_dict[doc_idx]
  print(doc_tfidf_encoded_vector)

print('Truy vấn được chuyển đổi về dạng tf-idf vector:')
print(query_tfidf_encoded_vector)

Các tài liệu được chuyển đổi về dạng tf-idf vectors:
[2.        4.        0.4150375 2.        0.4150375 0.4150375 2.
 2.        2.        2.        2.        0.        0.        0.
 0.        0.        0.        0.        0.        0.        0.
 0.        0.        0.        0.       ]
[1.        0.        0.4150375 0.        0.4150375 0.4150375 0.
 0.        0.        0.        0.        2.        2.        2.
 2.        2.        0.        0.        0.        0.        0.
 0.        0.        0.        0.       ]
[0.        0.        0.4150375 0.        0.4150375 0.4150375 0.
 0.        0.        0.        0.        0.        0.        0.
 0.        0.        2.        2.        2.        2.        2.
 2.        2.        2.        2.       ]
Truy vấn được chuyển đổi về dạng tf-idf vector:
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.]


**Tìm danh sách các tài liệu liên quan đến truy vấn (Tích vô hướng, khoảng cách Euclid và Tương đồng cosine)**

In [5]:
# Duyệt qua danh sách các tài liệu/văn bản (đã mã hóa ở dạng one-hot vectors)
for doc_idx in doc_id_tfidf_encoded_vector_dict.keys():
  doc_tfidf_encoded_vector = doc_id_tfidf_encoded_vector_dict[doc_idx]
  
  # Tính tích vô hướng giữa hai vectors tài liệu và truy vấn
  dot_product_sim = np.dot(query_tfidf_encoded_vector, doc_tfidf_encoded_vector)

  # Tính khoảng cách Euclid giữa hai vectors tài liệu và truy vấn
  ed = distance.euclidean(query_tfidf_encoded_vector, doc_tfidf_encoded_vector)
  
  # Tính tương đồ cosine giữa hai vectors tài liệu và truy vấn
  cs = 1 - distance.cosine(query_tfidf_encoded_vector, doc_tfidf_encoded_vector)

  print('Tài liệu: [{}], tương đồng (dot product): [{:.6f}]'.format(doc_idx, dot_product_sim))
  print('Tài liệu: [{}], tương đồng (khoảng cách Euclid): [{:.6f}]'.format(doc_idx, ed))
  print('Tài liệu: [{}], tương đồng (Tương đồng cosine): [{:.6f}]'.format(doc_idx, cs))
  print('---')

Tài liệu: [0], tương đồng (dot product): [0.000000]
Tài liệu: [0], tương đồng (khoảng cách Euclid): [6.820320]
Tài liệu: [0], tương đồng (Tương đồng cosine): [0.000000]
---
Tài liệu: [1], tương đồng (dot product): [4.000000]
Tài liệu: [1], tương đồng (khoảng cách Euclid): [3.939133]
Tài liệu: [1], tương đồng (Tương đồng cosine): [0.609757]
---
Tài liệu: [2], tương đồng (dot product): [0.000000]
Tài liệu: [2], tương đồng (khoảng cách Euclid): [6.206188]
Tài liệu: [2], tương đồng (Tương đồng cosine): [0.000000]
---
